# UT-ECE Data Science Final - Complete Solution Notebook (Q1-Q20)

This notebook provides executable solutions and concise answers for the full advanced assignment.
It covers core questions (Q1-Q6) and the production-grade extension block (Q15-Q20).


In [ ]:
from pathlib import Path
import sys

HERE = Path.cwd().resolve()
CODE_ROOT = None
for candidate in [HERE, HERE.parent, HERE.parent.parent]:
    if (candidate / 'scripts' / 'full_solution_pipeline.py').exists():
        CODE_ROOT = candidate
        break
    if (candidate / 'code' / 'scripts' / 'full_solution_pipeline.py').exists():
        CODE_ROOT = candidate / 'code'
        break

if CODE_ROOT is None:
    raise FileNotFoundError('Could not locate project code root. Run notebook from repo root or code/notebooks.')

sys.path.insert(0, str(CODE_ROOT / 'scripts'))

from full_solution_pipeline import (
    load_dataset,
    leakage_diagnostics,
    simulate_optimizers,
    plot_ravine_paths,
    run_q4_svm_and_pruning,
    run_q5_unsupervised,
    run_q6_capstone,
    run_q15_calibration_threshold,
    run_q16_drift_monitoring,
    run_q17_recourse_analysis,
    run_all,
)
from q18_temporal import run_q18_temporal_backtesting
from q19_uncertainty import run_q19_uncertainty_quantification
from q20_fairness_mitigation import run_q20_fairness_mitigation

DATA_PATH = CODE_ROOT / 'data' / 'GlobalTechTalent_50k.csv'
FIG_DIR = CODE_ROOT / 'figures'
SOL_DIR = CODE_ROOT / 'solutions'
PROFILE = 'balanced'
RANDOM_STATE = 42


In [ ]:
df = load_dataset(DATA_PATH)
print('Dataset shape:', df.shape)
print('Target distribution:')
print(df['Migration_Status'].value_counts(normalize=True).sort_index())
df.head(3)


## Q1 - Data Engineering and Leakage

**Q1A SQL answer:**
```sql
WITH citation_velocity AS (
    SELECT UserID, Country_Origin, Year, Research_Citations,
           AVG(Research_Citations) OVER (
               PARTITION BY Country_Origin
               ORDER BY Year
               ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
           ) AS moving_avg_citations
    FROM Professionals_Data
)
SELECT *, DENSE_RANK() OVER (
    PARTITION BY Country_Origin ORDER BY moving_avg_citations DESC
) AS country_rank
FROM citation_velocity;
```

**Q1B answer:** `Visa_Approval_Date` is direct leakage (post-outcome). `Last_Login_Region` and `Passport_Renewal_Status` can also be temporally leaky depending on collection timing.


In [ ]:
q1_diag = leakage_diagnostics(df)
q1_diag


## Q2 - Statistical Inference and Elastic Net

For
\[
J(\theta)=\frac{1}{2m}\sum_{i=1}^m(h_\theta(x^{(i)})-y^{(i)})^2 + \lambda_1\sum_j |\theta_j| + \frac{\lambda_2}{2}\sum_j \theta_j^2,
\]

the coordinate gradient is
\[
\nabla_{\theta_j}J = \frac{1}{m}\sum_i(h_\theta(x^{(i)})-y^{(i)})x_j^{(i)} + \lambda_1\partial|\theta_j| + \lambda_2\theta_j
\]
with subgradient \(\partial|\theta_j| \in [-1,1]\) at \(\theta_j=0\).

Given \(\beta=0.52\), \(p=0.003\), CI \([0.18,0.86]\): reject \(H_0: \beta=0\); the effect is statistically significant and positive.


## Q3 - Optimization (Ravine)

Momentum damps oscillations across steep ravine walls and accumulates velocity along consistent descent directions.
Adam further rescales steps using first/second moments, often converging faster under heterogeneous feature scales.


In [ ]:
paths = simulate_optimizers()
q3_info = plot_ravine_paths(paths, FIG_DIR / 'q3_ravine_optimizers.png')
q3_info


## Q4 - Non-Linear Models

If an RBF-SVM overfits, decrease `gamma` to widen support influence and smooth boundaries.
In post-pruning, larger `alpha` increases complexity penalty and shifts the bias-variance tradeoff toward lower variance.


In [ ]:
q4_info = run_q4_svm_and_pruning(df, FIG_DIR)
q4_info


## Q5 - Unsupervised Learning

In PCA, each eigenvalue gives variance captured by its principal direction. Explained variance ratio is
\(\lambda_k / \sum_i \lambda_i\).

For KMeans, WCSS decreases monotonically with K but exhibits diminishing returns, motivating the elbow heuristic.


In [ ]:
q5_info = run_q5_unsupervised(df, FIG_DIR)
q5_info


## Q6 - Capstone Explainability (SHAP)

`base_value` is the expected model output over background data.
`output_value` is the candidate-specific prediction.
Their difference equals the sum of local SHAP contributions.


In [ ]:
q6_info = run_q6_capstone(df, FIG_DIR, SOL_DIR)
q6_info


## Q15 - Calibration and Threshold Policy

This block evaluates probability reliability (Brier/ECE) and derives a decision threshold with explicit utility tradeoff.


In [ ]:
q15_info = run_q15_calibration_threshold(df, FIG_DIR)
q15_info


## Q16 - Drift Monitoring

Population Stability Index (PSI) is used to detect distribution drift between reference and monitoring windows.


In [ ]:
q16_info = run_q16_drift_monitoring(df, FIG_DIR, SOL_DIR)
q16_info


## Q17 - Recourse Analysis

Counterfactual recourse quantifies how many negative predictions can be flipped under bounded, realistic feature interventions.


In [ ]:
q17_info = run_q17_recourse_analysis(df, FIG_DIR, SOL_DIR)
q17_info


## Q18 - Temporal Backtesting and Performance Decay

Rolling backtesting estimates stability across time and measures whether discrimination power decays under temporal drift.


In [ ]:
q18_info = run_q18_temporal_backtesting(
    df,
    figures_dir=FIG_DIR,
    solutions_dir=SOL_DIR,
    profile=PROFILE,
    random_state=RANDOM_STATE,
)
q18_info


## Q19 - Uncertainty Quantification (Conformal)

Conformal prediction intervals target calibrated coverage across confidence levels and expose under-coverage risk.


In [ ]:
q19_info = run_q19_uncertainty_quantification(
    df,
    figures_dir=FIG_DIR,
    solutions_dir=SOL_DIR,
    profile=PROFILE,
    random_state=RANDOM_STATE,
)
q19_info


## Q20 - Fairness Mitigation Experiment

This section compares pre/post mitigation fairness gaps under an explicit policy guardrail on performance degradation.


In [ ]:
q20_info = run_q20_fairness_mitigation(
    df,
    figures_dir=FIG_DIR,
    solutions_dir=SOL_DIR,
    profile=PROFILE,
    random_state=RANDOM_STATE,
)
q20_info


## Full Run (Reproducible End-to-End)

Run the complete pipeline and regenerate all outputs (`SQL`, figures, markdown keys, JSON summaries, and LaTeX metrics).


In [ ]:
summary = run_all(
    DATA_PATH,
    FIG_DIR,
    SOL_DIR,
    profile=PROFILE,
    enable_q18=True,
    enable_q19=True,
    enable_q20=True,
)
summary


### Deliverables Produced

- `code/solutions/q1_moving_average.sql`
- `code/solutions/complete_solution_key.md`
- `code/solutions/run_summary.json`
- `code/solutions/latex_metrics.json`
- `code/solutions/latex_metrics.tex`
- `code/solutions/q18_temporal_backtest.csv`
- `code/solutions/q19_uncertainty_coverage.csv`
- `code/solutions/q20_fairness_mitigation_comparison.csv`
- `code/figures/q3_ravine_optimizers.png`
- `code/figures/q4_svm_gamma_sweep.png`
- `code/figures/q4_tree_pruning_curve.png`
- `code/figures/q5_kmeans_elbow.png`
- `code/figures/q6_shap_force_plot.png`
- `code/figures/q6_shap_summary.png`
- `code/figures/q15_calibration_curve.png`
- `code/figures/q15_threshold_tradeoff.png`
- `code/figures/q16_drift_top_psi.png`
- `code/figures/q17_recourse_summary.png`
- `code/figures/q18_temporal_degradation.png`
- `code/figures/q19_coverage_vs_alpha.png`
- `code/figures/q20_fairness_tradeoff.png`
